In [2]:
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

## e005 の valid を確認する

In [3]:
trn_df = pd.read_csv('../inputs/origin/train.csv')
display(trn_df.shape, trn_df.head())

(27481, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
ls ../checkpoints/e005/0/

epoch_0_9.17545_-1.00000_0.51232_checkpoint.pth
epoch_1_1.62222_-1.00000_0.68830_checkpoint.pth
epoch_2_1.51474_-1.00000_0.69574_checkpoint.pth


In [5]:
ckpt = torch.load('../checkpoints/e005/0/epoch_2_1.51474_-1.00000_0.69574_checkpoint.pth')

In [5]:
ckpt.keys()

dict_keys(['fold_num', 'current_epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'val_textIDs', 'val_input_ids', 'val_preds', 'val_labels', 'histories'])

In [6]:
trn_df = trn_df.query(f"textID in {list(ckpt['val_textIDs'])}")
trn_df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive
11,16fab9f95b,I really really like the song Love Story by Ta...,like,positive
...,...,...,...,...
27467,432e6de6c9,morning twit-friends! welcome to my new followers,welcome,positive
27469,ac92790d8b,94 more days till BH comes back to LA,94 more days till BH comes back to LA,neutral
27472,8f5adc47ec,http://twitpic.com/663vr - Wanted to visit the...,were too late,negative
27474,8f14bb2715,So I get up early and I feel good about the da...,I feel good ab,positive


In [7]:
def _get_predicted_texts(input_ids, y_preds_head, y_preds_tail, tokenizer):
    predicted_texts = []
    for input_id, y_pred_head, y_pred_tail \
            in zip(input_ids, y_preds_head, y_preds_tail):
        pred_label_head = y_pred_head.argmax()
        pred_label_tail = y_pred_tail.argmax()
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [8]:
def _get_selected_texts(input_ids, labels_head, labels_tail, tokenizer):
    predicted_texts = []
    for input_id, pred_label_head, pred_label_tail \
            in zip(input_ids, labels_head, labels_tail):
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [22]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [23]:
predicted_texts = _get_predicted_texts(ckpt['val_input_ids'], ckpt['val_preds'][0],  ckpt['val_preds'][1], tokenizer)
trn_df['predicted_texts'] = predicted_texts
trn_df

,textID,text,selected_text,sentiment,predicted_texts,selected_text_lower,manual_selected_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going","i`d have responded, if i were going",[unused934] extant [unused412] [unused32] loca...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,leave me alone,leave me alone,[unused984] [unused157] 耳
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****,","sons of ****,",ballet [unused8] [UNK] [unused5]
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive,wow... u just became cooler.,wow... u just became cooler.,##orough [unused729] ノ [unused94] w judith [un...
11,16fab9f95b,I really really like the song Love Story by Ta...,like,positive,i really really like,like,[CLS]
...,...,...,...,...,...,...,...
27467,432e6de6c9,morning twit-friends! welcome to my new followers,welcome,positive,welcome,welcome,friends
27469,ac92790d8b,94 more days till BH comes back to LA,94 more days till BH comes back to LA,neutral,94 more days till bh comes back to la,94 more days till bh comes back to la,burial [unused54] [unused355] clinical [unused...
27472,8f5adc47ec,http://twitpic.com/663vr - Wanted to visit the...,were too late,negative,too late,were too late,[unused57] [unused345] [unused623]
27474,8f14bb2715,So I get up early and I feel good about the da...,I feel good ab,positive,good,i feel good ab,[unused934] [unused614] [unused200] [unused58]


In [24]:
manual_selected_texts = _get_selected_texts(ckpt['val_input_ids'], ckpt['val_labels'][0],  ckpt['val_labels'][1], tokenizer)
trn_df['manual_selected_text'] = manual_selected_texts
trn_df

,textID,text,selected_text,sentiment,predicted_texts,selected_text_lower,manual_selected_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going"
3,9642c003ef,what interview! leave me alone,leave me alone,negative,leave me alone,leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****,","sons of ****,","sons of ****,"
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive,wow... u just became cooler.,wow... u just became cooler.,wow... u just became cooler.
11,16fab9f95b,I really really like the song Love Story by Ta...,like,positive,i really really like,like,like
...,...,...,...,...,...,...,...
27467,432e6de6c9,morning twit-friends! welcome to my new followers,welcome,positive,welcome,welcome,welcome
27469,ac92790d8b,94 more days till BH comes back to LA,94 more days till BH comes back to LA,neutral,94 more days till bh comes back to la,94 more days till bh comes back to la,94 more days till bh comes back to la
27472,8f5adc47ec,http://twitpic.com/663vr - Wanted to visit the...,were too late,negative,too late,were too late,were too late
27474,8f14bb2715,So I get up early and I feel good about the da...,I feel good ab,positive,good,i feel good ab,i feel good about


In [14]:
ckpt['val_labels'][0][0],  ckpt['val_labels'][1][0]

(tensor(1), tensor(11))

In [16]:
ckpt['val_input_ids'][0]

tensor([    0,   939, 12905,   417,    33,  2334,     6,   114,   939,    58,
          164,     2,     2, 50265,     2,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1])

In [61]:
tokenizer.encode_plus(trn_df.loc[11]['selected_text'])

{'input_ids': [101, 2066, 102],
 'token_type_ids': [0, 0, 0],
 'attention_mask': [1, 1, 1]}

In [78]:
tokenizer.encode_plus(trn_df.loc[11]['selected_text'], add_special_tokens=False)['input_ids']

[2066]

## debug

In [12]:
def _prep_text(row, tokenizer):
    text_output = tokenizer.encode_plus(
        text=row['text'],
        text_pair=None,
        add_special_tokens=True,
        max_length=250,
        pad_to_max_length=True,
        return_tensor='pt',
        return_token_type_ids=False,
        return_attention_mask=True,
    )
    row['input_ids'] = text_output['input_ids']
    row['attention_mask'] = text_output['attention_mask']
    if 'selected_text' not in row:
        row['labels_head'] = -1
        row['labels_tail'] = -1
        return row

    selected_text_output = tokenizer.encode_plus(
        text=row['selected_text'],
        text_pair=None,
        add_special_tokens=False,
        max_length=250,
        pad_to_max_length=False,
        return_tensor='pt',
        return_token_type_ids=False,
        return_attention_mask=False,
    )

    # allign labels for segmentation
    input_ids = text_output['input_ids']
    sel_input_ids = selected_text_output['input_ids']
    matched_cnt = len([i for i in input_ids[:len(sel_input_ids)]
                       if i in sel_input_ids])
    best_matched_cnt = matched_cnt
    best_matched_i = 0
    # for i in range(0, len(input_ids)):
    for i in range(0, len(input_ids) - len(sel_input_ids)):
        head_input_id_i = input_ids[i]
        tail_input_id_i = input_ids[i + len(sel_input_ids)]
        if head_input_id_i in sel_input_ids:
            matched_cnt -= 1
        if tail_input_id_i in sel_input_ids:
            matched_cnt += 1
        if matched_cnt < 0:
            raise Exception('invalid logic')

        if best_matched_cnt < matched_cnt:
            best_matched_cnt = matched_cnt
            best_matched_i = i + 1   # $BH4$$$?;~$NOC$J$N$G(B
        if best_matched_cnt == len(sel_input_ids):
            break

    if best_matched_cnt == 0:
        print('===============================')
        print(row)
        print('===============================')
        print(input_ids)
        print(sel_input_ids)
        print(selected_text_output['input_ids'])

    row['labels_head'] = best_matched_i
    row['labels_tail'] = best_matched_i + len(sel_input_ids)
    return row

In [13]:
res_row = _prep_text(trn_df.loc[11], tokenizer)
res_row

textID                                                         16fab9f95b
text                    I really really like the song Love Story by Ta...
selected_text                                                        like
sentiment                                                        positive
predicted_texts                                                love story
manual_selected_text                                             like the
input_ids               [101, 1045, 2428, 2428, 2066, 1996, 2299, 2293...
attention_mask          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...
labels_head                                                             4
labels_tail                                                             5
Name: 11, dtype: object

## tokenizer 確認

In [17]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

my_tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

In [21]:
my_tokenizer.decode(ckpt['val_input_ids'][0][1:11])

' i`d have responded, if i were going'

In [52]:
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [53]:
temp_jac = 0
for i, row in trn_df.iterrows():
    temp_jac += jaccard(row['selected_text'], row['predicted_texts'])

In [54]:
temp_jac / len(trn_df)

0.44581498228808364

## 改善の必要あり
 - decode すると違う形式になってしまう
 - selected_text を encode -> decode して形式が変わるパターンを除去する後処理が必要そう

In [75]:
trn_df.loc[0]

textID                                                         cb774db0d1
text                                  I`d have responded, if I were going
selected_text                         I`d have responded, if I were going
sentiment                                                         neutral
predicted_texts         i ` d have responded, if i were going [SEP] [PAD]
manual_selected_text    i ` d have responded, if i were going [SEP] [PAD]
Name: 0, dtype: object

In [74]:
jaccard(trn_df.loc[0]['selected_text'], trn_df.loc[0]['predicted_texts'][:-12])

0.6666666666666666

In [72]:
trn_df.loc[0]['selected_text'][4:], trn_df.loc[0]['predicted_texts'][6:-12]

('have responded, if I were going', 'have responded, if i were going')